# Import preprocessed data

In [1]:
from tqdm import tqdm
import numpy as np
import mmh3

In [2]:
import import_ipynb
from loading_and_preprocessing import movies, users

importing Jupyter notebook from loading_and_preprocessing.ipynb
User(1488844)
ratings on the format movie_id : rating  defaultdict(<class 'float'>, {'1': 3, '8': 4, '17': 2, '30': 3})
Dinosaur Planet, 2003
ratings on the format user_id : rating:  {'1488844': 3, '822109': 5, '885013': 4, '30878': 4, '823519': 3, '893988': 3, '124105': 4, '1248029': 3, '1842128': 4, '2238063': 3, '1503895': 4, '2207774': 5, '2590061': 3, '2442': 3, '543865': 4, '1209119': 4, '804919': 4, '1086807': 3, '1711859': 4, '372233': 5, '1080361': 3, '1245640': 3, '558634': 4, '2165002': 4, '1181550': 3, '1227322': 4, '427928': 4, '814701': 5, '808731': 4, '662870': 5, '337541': 5, '786312': 3, '1133214': 4, '1537427': 4, '1209954': 5, '2381599': 3, '525356': 2, '1910569': 4, '2263586': 4, '2421815': 2, '1009622': 1, '1481961': 2, '401047': 4, '2179073': 3, '1434636': 3, '93986': 5, '1308744': 5, '2647871': 4, '1905581': 5, '2508819': 3, '1578279': 1, '1159695': 4, '2588432': 3, '2423091': 3, '470232': 4, '214869

# Compute Signature matrix

In [3]:
movies

defaultdict(src.structures.Movie,
            {'1': Movie(1, Dinosaur Planet, 2003) seen by 547 users,
             '2': Movie(2, Isle of Man TT 2004 Review, 2004) seen by 145 users,
             '3': Movie(3, Character, 1997) seen by 2012 users,
             '4': Movie(4, Paula Abdul's Get Up & Dance, 1994) seen by 142 users,
             '5': Movie(5, The Rise and Fall of ECW, 2004) seen by 1140 users,
             '6': Movie(6, Sick, 1997) seen by 1019 users,
             '7': Movie(7, 8 Man, 1992) seen by 93 users,
             '8': Movie(8, What the #$*! Do We Know!?, 2004) seen by 14910 users,
             '9': Movie(9, Class of Nuke 'Em High 2, 1991) seen by 95 users,
             '10': Movie(10, Fighter, 2001) seen by 249 users,
             '11': Movie(11, Full Frame: Documentary Shorts, 1999) seen by 198 users,
             '12': Movie(12, My Favorite Brunette, 1947) seen by 546 users,
             '13': Movie(13, Lord of the Rings: The Return of the King: Extended Edition: B

In [4]:
#some choice of minhash function
def minhash_function(id, rating, seed):
    hashval = mmh3.hash(str(id), seed = seed) ^ mmh3.hash(str(rating), seed = seed)
    return hashval

def signature(id, rating, n_minhash = 100): 
    return [minhash_function(id, rating, seed) for seed in range(n_minhash)]

def bucket_hash(signatures, n_buckets = 100):
    bucket = 0
    for signature in signatures:
        bucket = bucket ^ mmh3.hash(str(signature)) 
    return bucket % n_buckets

In [5]:
from collections import defaultdict
import numpy as np

def user_SIG_matrix(users, n_hashes = 100):
    #initialize signature matrix with infinity
    SIG = defaultdict(lambda: np.inf*np.ones(n_hashes))
    
    for user_id, user in users.items():
        for movie_id, rating in user.ratings.items():
            hashes = signature(movie_id, rating, n_hashes)
            SIG[user_id] = np.minimum(SIG[user_id], hashes)
    return SIG

In [12]:
SIG = user_SIG_matrix(users)

In [11]:
SIG

defaultdict(<function __main__.user_SIG_matrix.<locals>.<lambda>()>,
            {'1488844': array([-1.23586166e+09, -1.46240512e+09, -1.16782572e+09, -1.75246766e+09,
                    -1.62546222e+09, -1.42159105e+09, -1.13678170e+09, -2.70295808e+08,
                    -4.66315440e+08, -6.84055932e+08, -3.58705068e+08, -1.39442714e+09,
                    -9.75855340e+08,  2.70516534e+08, -1.88919878e+09, -1.22883180e+09,
                    -1.34600354e+09, -1.32325124e+09, -1.24323746e+09, -1.87877866e+09,
                    -1.89103410e+09, -2.11259411e+09,  4.14076319e+08, -1.97597355e+09,
                    -1.15379612e+09, -8.24899313e+08, -1.77511969e+09, -2.11800861e+09,
                    -1.64937170e+09, -7.36813119e+08, -1.27658024e+09, -1.60724702e+09,
                    -1.43738314e+09, -1.06492638e+09, -5.93674208e+08, -1.47833325e+09,
                    -1.13190726e+09, -1.69700118e+09, -1.80589775e+09,  6.60845664e+08,
                    -1.38765030e+09, -9.

In [7]:
# n_hashes = 100
# def locality_sensitive_hashing(SIG, n_hashes = 100, bands = 10, n_buckets = 10000):
#     r = n_hashes // bands
#     similar = defaultdict(set)
#     for i in tqdm(range(0, n_hashes, r)):
#         buckets = defaultdict(list)
#         for user_id, sig in SIG.items():
#             bucket = bucket_hash(sig[i:i+r], n_buckets=n_buckets)

#             if buckets[bucket]:
#                 for prev_user_id in buckets[bucket]:
#                     similar[user_id].add(prev_user_id)
#                     similar[prev_user_id].add(user_id)
#             buckets[bucket].append(user_id)
            
#     return similar

In [8]:
# similar = locality_sensitive_hashing(SIG, n_hashes = n_hashes, bands = 20, n_buckets = 10000000)

In [13]:
Sim_matrix = np.zeros((len(users), len(users)))
keys = list(users.keys())
for i in tqdm(range(len(users))):
    for j in range(i, len(users)):
        if i > j:
            Sim_matrix[i][j] = users[keys[i]].similarity(users[keys[j]])

MemoryError: Unable to allocate 49.5 GiB for an array with shape (81472, 81472) and data type float64

In [119]:
users

defaultdict(src.structures.User,
            {'1488844': User(1488844) has rated 2 movies,
             '822109': User(822109) has rated 1 movies,
             '885013': User(885013) has rated 2 movies,
             '30878': User(30878) has rated 2 movies,
             '823519': User(823519) has rated 2 movies,
             '893988': User(893988) has rated 1 movies,
             '124105': User(124105) has rated 1 movies,
             '1248029': User(1248029) has rated 1 movies,
             '1842128': User(1842128) has rated 1 movies,
             '2238063': User(2238063) has rated 1 movies,
             '1503895': User(1503895) has rated 1 movies,
             '2207774': User(2207774) has rated 1 movies,
             '2590061': User(2590061) has rated 1 movies,
             '2442': User(2442) has rated 1 movies,
             '543865': User(543865) has rated 1 movies,
             '1209119': User(1209119) has rated 1 movies,
             '804919': User(804919) has rated 1 movies,
     